<a href="https://colab.research.google.com/github/msugar/zoomcamp/blob/main/data-engineering-2024/workshops/01-dlt/Homework_data_talks_club_data_extraction_and_ingestion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [1]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [2]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)

{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [3]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [4]:
# to do: homework :)

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX

**Question 1**

In [4]:
print("Question 1. What is the sum of the outputs of the generator for limit = 5?")
generator = square_root_generator(5)
answer = sum(generator)
print(f"{answer=}")

Question 1. What is the sum of the outputs of the generator for limit = 5?
answer=8.382332347441762


**Question 2**

In [5]:
from collections import deque

print("Question 2. What is the 13th number yielded by the generator?")
generator = square_root_generator(13)
dd = deque(generator, maxlen=1)
last_element = dd.pop()
print(f"{last_element=}")

Question 2. What is the 13th number yielded by the generator?
last_element=3.605551275463989


**Question 3**

In [6]:
from itertools import chain

print("Question 3. Append the 2 generators. After correctly appending the data, calculate the sum of all ages of people.")
people = chain(people_1(), people_2())
ages_sum = sum(person['Age'] for person in people)
print(f"{ages_sum=}")

Question 3. Append the 2 generators. After correctly appending the data, calculate the sum of all ages of people.
ages_sum=353


**Duck DB**

In [7]:
import dlt

import duckdb
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [20]:
# define the connection to load to.
pipeline = dlt.pipeline(pipeline_name="people_pipeline", destination="duckdb", dataset_name="people_data")

# Normalize and load the data onto the locally created duckdb database 'pokemon_pipeline.duckdb'
info = pipeline.run(people_1(), table_name='people')
print(info)

Pipeline people_pipeline load step completed in 0.24 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_data
The duckdb destination used duckdb:////content/people_pipeline.duckdb location to store data
Load package 1707882035.4406438 is LOADED and contains no failed jobs


In [21]:
# a database 'people_pipeline.duckdb' was created in working directory so just connect to it
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# this lets us query data without adding schema prefix to table names
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")

# list all tables
display(conn.sql("DESCRIBE"))

# show the outcome
people_table = conn.sql("SELECT * FROM people ORDER BY id").df()
display(people_table)

┌─────────────────┬─────────────────────┬─────────────────────┬───┬──────────────────────┬───────────┐
│    database     │       schema        │        name         │ … │     column_types     │ temporary │
│     varchar     │       varchar       │       varchar       │   │      varchar[]       │  boolean  │
├─────────────────┼─────────────────────┼─────────────────────┼───┼──────────────────────┼───────────┤
│ people_pipeline │ people_data         │ _dlt_loads          │ … │ [VARCHAR, VARCHAR,…  │ false     │
│ people_pipeline │ people_data         │ _dlt_pipeline_state │ … │ [BIGINT, BIGINT, V…  │ false     │
│ people_pipeline │ people_data         │ _dlt_version        │ … │ [BIGINT, BIGINT, T…  │ false     │
│ people_pipeline │ people_data         │ people              │ … │ [BIGINT, VARCHAR, …  │ false     │
│ people_pipeline │ people_data_staging │ _dlt_version        │ … │ [BIGINT, BIGINT, T…  │ false     │
│ people_pipeline │ people_data_staging │ people              │ … │ [BIGI

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707882035.4406438,Fc0faTEW8YUdXw,None
1,2,Person_2,27,City_A,1707882035.4406438,Vlnd4rKOEZLK6A,None
2,3,Person_3,28,City_A,1707882035.4406438,FI7ue4tQsZZaWw,None
3,4,Person_4,29,City_A,1707882035.4406438,8feX8TcR7UW/VA,None
4,5,Person_5,30,City_A,1707882035.4406438,hDLBEMm4bW0kkw,None


In [22]:
# Merge
info = pipeline.run(people_2(),
										table_name="people",
										write_disposition="merge",
                    primary_key='ID')
print(info)

# show the outcome
people_table = conn.sql("SELECT * FROM people ORDER BY id").df()
display(people_table)

Pipeline people_pipeline load step completed in 0.33 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_data
The duckdb destination used duckdb:////content/people_pipeline.duckdb location to store data
Load package 1707882054.1684337 is LOADED and contains no failed jobs


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707882035.4406438,Fc0faTEW8YUdXw,None
1,2,Person_2,27,City_A,1707882035.4406438,Vlnd4rKOEZLK6A,None
2,3,Person_3,33,City_B,1707882054.1684337,boXT0UluxGN67w,Job_3
3,4,Person_4,34,City_B,1707882054.1684337,8yglhlyyyCepSA,Job_4
4,5,Person_5,35,City_B,1707882054.1684337,lTUkhsbTg/sG7Q,Job_5
5,6,Person_6,36,City_B,1707882054.1684337,hZcvpMvCFcxXKg,Job_6
6,7,Person_7,37,City_B,1707882054.1684337,TGQfe/l2AMC90w,Job_7
7,8,Person_8,38,City_B,1707882054.1684337,R7YFa8Y0Lm7CYQ,Job_8


**Question 4**

In [23]:
print("Question 4. Merge the 2 generators using the ID column. Calculate the sum of ages of all the people loaded as described above.")
merged_all_sum = conn.sql("SELECT SUM(age) FROM people")
display(merged_all_sum)

Question 4. Merge the 2 generators using the ID column. Calculate the sum of ages of all the people loaded as described above.


┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      266 │
└──────────┘

**Clean Up**

In [24]:
conn.sql("DELETE FROM people")